In [5]:
import pickle
import numpy as np
from multiprocessing import Pool
import requests, json
import pandas as pd
import glob
import timeit
import time
from multiprocessing import Pool
import numpy as np
def toc(start_time):
    elapsed = timeit.default_timer() - start_time
    print(elapsed)
from zipfile import ZipFile
import re
import xmlschema
from pprint import pprint
import glob
# importing element tree
import lxml.etree as etree
import pickle


# country codes : all vs EU

In [38]:
my_file = open("all_country_codes.txt", "r")
all_country_codes = my_file.read()
# print(all_country_codes)

all_country_codes = all_country_codes.split("\n")[1:]
all_country_codes = [x.split(",") for x in all_country_codes]
all_country_codes


my_file = open("eu_codes.txt", "r")
eu_codes = my_file.read()
eu_codes = eu_codes.split("\n")
eu_codes = [x.split("-") for x in eu_codes]
a=[]
for x in eu_codes:
    b=[]
    for y in x:
        b.append(y.strip())
    a.append(b)
eu_codes = [x[0:2] for x in eu_codes]
eu_codes_only=[x[1].strip() for x in eu_codes]

# print(all_country_codes)

In [43]:
all_country_codes_only=[x[1].strip() for x in all_country_codes]
len([x for x in all_country_codes_only if x not in eu_codes_only])

222

In [45]:
len(all_country_codes_only)
len(eu_codes_only)

27

# NSF grant years for each scientist

In [6]:
path_career='/mnt/sdb1/sandeep/5. NSF vs ERC/data/'

with open(path_career+'dict_name_to_info_NSF.pkl', 'rb') as f:
    dict_name_to_info_NSF=pickle.load(f)    


In [7]:
list_dict=[]
import pandas as pd

for year in dict_name_to_info_NSF.keys():
    scientists_all_nsf=list(dict_name_to_info_NSF[year].keys())
#     for it_award in range(IT_,IT_+100):
    for it_award in range(len(scientists_all_nsf)):
        xx=dict()
        award=dict_name_to_info_NSF[year][scientists_all_nsf[it_award]]
#         print(len(award))
#         print(award[0].keys())
        xx['scientist']=scientists_all_nsf[it_award].replace(',','')
        xx['start_date']=award[0]['StartDate']
        xx['end_date']=award[0]['AwardExpirationDate']
        xx['type']=award[0]['Value']
        list_dict.append(xx)
        
#         print(scientists_all_nsf[it_award])
#         print(award[0]['AwardEffectiveDate'])
#         print(award[0]['AwardExpirationDate'],'\n')        
#         dict_sci_start_end_dates[scientists_all_nsf[it_award].replace(',','')]=    
#         dict_sci_award_type[scientists_all_nsf[it_award]]=
    print('year=',year,end='\r')

In [51]:
df_NSF_grant_dates= pd.DataFrame(list_dict)


In [52]:
df_NSF_grant_dates.head()

,scientist,start_date,end_date,type
0,Alan Goldman,10/11/1974,09/11/1977,Interagency Agreement
1,Julius Tou,05/01/1975,04/30/1978,Standard Grant
2,Herbert F York,10/01/1974,03/31/1977,Standard Grant
3,John Fetkovich,07/01/1975,10/31/1976,Standard Grant
4,D. P Gregory,05/01/1975,06/01/1976,Contract


In [133]:
# df_NSF[df_NSF['scientist']=='Ponisseril Somasundaran']
# df_NSF[df_NSF['scientist']=='Thomas Jordan']
df_NSF.groupby('scientist').count().sort_values('start_date',ascending=False).iloc[0:10]

# Collaborations patterns of NSF winners

In [202]:
# import glob
# list_NSF_collab_patterns={}

# path_career='/mnt/sdb1/sandeep/5. NSF vs ERC/data/'

# # for year in range()
# year=str(2010)
# # files_=glob.glob(path_career+'dict_NSF_careers_*')
# files_=glob.glob(path_career+'dict_NSF_careers_'+year+'*')
# #-----------------------------------------
# dict_careers={}
# for file in files_:
#     with open(file, 'rb') as f:
#         dict_careers[file.split('/')[-1]]=pickle.load(f)                
        

In [201]:
path_codes='/mnt/sdb1/sandeep/5. NSF vs ERC/codes/'
path_data='/mnt/sdb1/sandeep/5. NSF vs ERC/data/'
dict_list_NSF_collab_patterns={}
dfs_NSF={}

for year_int in range(2010,2021):
    year=str(year_int)
    print(year,end='\r')
    with open(path_data+'dict_list_NSF_collab_patterns_'+year+'.pkl', 'rb') as f:
        dict_list_NSF_collab_patterns[year]=pickle.load(f)   
    dfs_NSF[year]= pd.DataFrame(dict_list_NSF_collab_patterns[year])
    dfs_NSF[year]=dfs_NSF[year].sort_values(['scientist','pub_year'],ascending=True)
    dfs_NSF[year]=dfs_NSF[year][dfs_NSF[year]['scientist']!='- Robby']
    countries_temp=list(dfs_NSF[year]['countries'])

    n_identified=[len(list(filter(None, x))) for x in countries_temp]
    dfs_NSF[year]['identified_countries']=n_identified
    dfs_NSF[year]['n_US']=[np.sum(np.array(x)=='US') for x in countries_temp]
    progress='progress'+'--year--   '+str(year)+'--n_US done'
    print(progress,end='\r')
    with open(path_codes+"2.nsf_tracker.txt", "w") as file_object:
        file_object.write(progress+'\n')
    
    dfs_NSF[year]['n_EU']=[np.sum(np.in1d(x,eu_codes_only)) for x in countries_temp]        
    
    progress='progress'+'--year--   '+str(year)+'--n_EU done'
    print(progress,end='\r')
    with open(path_codes+"2.nsf_tracker.txt", "w") as file_object:
        file_object.write(progress+'\n')
    
    dfs_NSF[year]['subtract_from_n_US']=(dfs_NSF[year]['country_awardee']=='US').astype('int')
    
    progress='progress'+'--year--   '+str(year)+'--finished'
    print(progress,end='\r')
    with open(path_codes+"2.nsf_tracker.txt", "w") as file_object:
        file_object.write(progress+'\n')


<ipython-input-201-faea03a59584>:18: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  dfs_NSF[year]['n_US']=[np.sum(np.array(x)=='US') for x in countries_temp]


In [ ]:
        
with open(path_data+'dfs_NSF.pkl', 'wb') as f:
    pickle.dump(dfs_NSF, f)


In [188]:
df_NSF[df_NSF['n_EU']>0].shape[0]/df_NSF.shape[0]

0.0934820494003853

In [189]:
df_NSF[df_NSF['n_EU']>0]

,scientist,countries,team_size,pub_year,counts_by_year,country_awardee,identified_countries,n_US,subtract_from_n_US,n_EU
1184399,A Joshua West,"[DE, US, GB, DE, FR, DE, CA, None]",8,2013,"[{'year': 2022, 'cited_by_count': 18}, {'year'...",US,7,1,1,4
1184392,A Joshua West,"[None, DE, None, CN, CN, GB, GB]",7,2014,"[{'year': 2022, 'cited_by_count': 5}, {'year':...",US,5,0,1,1
1184376,A Joshua West,"[US, US, ES, ES, ES, None, ES, ES, ES, ES, Non...",15,2015,"[{'year': 2022, 'cited_by_count': 1}, {'year':...",US,11,3,1,8
1184362,A Joshua West,"[US, US, US, US, FR, US, US, US, US]",9,2016,"[{'year': 2021, 'cited_by_count': 18}, {'year'...",US,9,8,1,1
1184363,A Joshua West,"[CN, DE, US, US, TW, CN, None, CN, US, CN, CN,...",16,2016,"[{'year': 2022, 'cited_by_count': 3}, {'year':...",US,15,3,1,1
...,...,...,...,...,...,...,...,...,...,...
386639,richard scott,"[GB, GB, ES, CN, US, US, US, US, GB, None, Non...",21,2021,"[{'year': 2022, 'cited_by_count': 7}, {'year':...",US,17,6,1,2
386646,richard scott,"[US, US, US, ES, US, US]",6,2021,[],US,6,5,1,1
386648,richard scott,"[US, US, US, US, US, US, US, US, US, DE]",10,2021,[],US,10,9,1,1
386667,richard scott,"[FR, US, FR, FR]",4,2021,"[{'year': 2022, 'cited_by_count': 3}, {'year':...",US,4,1,1,3


### NSF creating df_career_n_grant

In [190]:
df_NSF_grant_dates.head()

,scientist,start_date,end_date,type
0,Alan Goldman,10/11/1974,09/11/1977,Interagency Agreement
1,Julius Tou,05/01/1975,04/30/1978,Standard Grant
2,Herbert F York,10/01/1974,03/31/1977,Standard Grant
3,John Fetkovich,07/01/1975,10/31/1976,Standard Grant
4,D. P Gregory,05/01/1975,06/01/1976,Contract


In [194]:
df_NSF_grant_dates_standard=df_NSF_grant_dates[df_NSF_grant_dates.type=='Standard Grant']
df_NSF_grant_dates_standard

,scientist,start_date,end_date,type
1,Julius Tou,05/01/1975,04/30/1978,Standard Grant
2,Herbert F York,10/01/1974,03/31/1977,Standard Grant
3,John Fetkovich,07/01/1975,10/31/1976,Standard Grant
5,A. F. Spilhaus Jr.,04/01/1975,03/31/1979,Standard Grant
6,John White,01/01/1976,06/30/1978,Standard Grant
...,...,...,...,...
435379,Jennifer Fehrenbacher,05/09/2022,05/31/2025,Standard Grant
435380,Dev Niyogi,05/03/2022,11/30/2022,Standard Grant
435381,Gregory B Thompson,05/12/2022,05/31/2023,Standard Grant
435382,Bruce Landman,05/11/2022,06/30/2022,Standard Grant


In [181]:
# countries_temp

# ERC grants by df

In [3]:

with open(path_career+'df_ERC_scientist_award_details.pkl', 'rb') as f:
    df_ERC_scientist_award_details=pickle.load(f)    


In [4]:
df_ERC_scientist_award_details

,Project,Researcher (PI),Host Institution (HI),Country,Call Details,Summary,Summary\n,Max ERC Funding,Duration,starting_grant,adv_grant,cons_grant,proof_grant,synergy_grant
0,Spintronics based on relativistic phenomena in...,Tomá Jungwirth,FYZIKALNI USTAV AV CR V.V.I,Czechia,"Advanced Grant (AdG), PE3, ERC-2010-AdG_20100224",The 0MSPIN project consists of an extensive in...,The 0MSPIN project consists of an extensive in...,1 938 000 €\n,"Start date: 2011-06-01, End date: 2016-05-31\n",0,1,0,0,0
1,Genome sequences from extinct hominins,Svante PÄÄBO,MAX-PLANCK-GESELLSCHAFT ZUR FORDERUNG DER WISS...,Germany,"Advanced Grant (AdG), LS2, ERC-2015-AdG","Neandertals and Denisovans, an Asian group dis...","Neandertals and Denisovans, an Asian group dis...",2 350 000 €\n,"Start date: 2016-11-01, End date: 2021-10-31\n",0,1,0,0,0
2,Towards Nanostructured Electrocatalysts with S...,Nejc HODNIK,KEMIJSKI INSTITUT,Slovenia,"Starting Grant (StG), PE4, ERC-2019-STG","In the last decades, significant progress has ...","In the last decades, significant progress has ...",1 496 750 €\n,"Start date: 2020-01-01, End date: 2024-12-31\n",1,0,0,0,0
3,Radiocarbon constraints for models of C cyclin...,Susan Trumbore,MAX-PLANCK-GESELLSCHAFT ZUR FORDERUNG DER WISS...,Germany,"Advanced Grant (AdG), PE10, ERC-2015-AdG",The overall goal of 14Constraint is to enhance...,The overall goal of 14Constraint is to enhance...,2 283 747 €\n,"Start date: 2016-12-01, End date: 2021-11-30\n",0,1,0,0,0
4,The 15th-century Book Trade: An Evidence-based...,Cristina Dondi,"THE CHANCELLOR, MASTERS AND SCHOLARS OF THE UN...",United Kingdom,"Consolidator Grant (CoG), SH6, ERC-2013-CoG",The idea that underpins this project is to use...,The idea that underpins this project is to use...,1 999 172 €\n,"Start date: 2014-04-01, End date: 2019-03-31\n",0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11977,Zooarchaeology and Mobility in the Western Med...,Silvia VALENZUELA LAMAS,AGENCIA ESTATAL CONSEJO SUPERIOR DEINVESTIGACI...,Spain,"Starting Grant (StG), SH6, ERC-2016-STG",Human survival and success is substantially de...,Human survival and success is substantially de...,1 199 319 €\n,"Start date: 2017-04-01, End date: 2022-01-31\n",1,0,0,0,0
11978,The Pancreas Regulome: From causality to predi...,José Carlos Ribeiro Bessa,INSTITUTO DE BIOLOGIA MOLECULAR E CELULAR-IBMC,Portugal,"Starting Grant (StG), LS2, ERC-2015-STG",Several human pancreatic diseases have been ch...,Several human pancreatic diseases have been ch...,1 497 520 €\n,"Start date: 2016-06-01, End date: 2021-05-31\n",1,0,0,0,0
11979,Hand pose and gesture analysis for next genera...,Ron Kimmel,TECHNION - ISRAEL INSTITUTE OF TECHNOLOGY,Israel,"Proof of Concept (PoC), PC1, ERC-2014-PoC","A touch-less, markerless interface for next ge...","A touch-less, markerless interface for next ge...",150 000 €\n,"Start date: 2015-06-01, End date: 2016-11-30\n",0,0,0,1,0
11980,Zygotic Cell Fate and Parent-Biased Gene Expre...,Aleksandar Vjestica,UNIVERSITE DE LAUSANNE,Switzerland,"Starting Grant (StG), LS3, ERC-2020-STG","As two gametes fuse, the newly formed zygote i...","As two gametes fuse, the newly formed zygote i...",1 702 705 €\n,"Start date: 2020-12-01, End date: 2025-11-30\n",1,0,0,0,0
